In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
from scipy.io import savemat, loadmat
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout, Dense, Activation
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import regularizers
from tensorflow.keras import metrics, losses
from tensorflow.keras import backend as K

You can use the following keras tutorial for sequential models to implement and train your network in this problem.

https://keras.io/getting-started/sequential-model-guide/

# Part 1

In [ ]:
# Load data
data = loadmat('P2_train.mat')
V = data['X']
Z = data['Y']

In [ ]:
# ToDo: perform PCA on the velocity and call the e-vector matrix U


In [ ]:
# ToDo: Calculate the reconstruction RMSE and plot it. If necessary, try using an informative plot
# such as semi-log or log-log

# Part 2

In [ ]:
# ToDo: number of components to obtain 10% of the error from Problem 1?

In [ ]:
# Setting up input and output of the network, X_train and Y_train,
# after performing proper pre-training normalization of the data.
# ToDo: You have to set l equal to the proper number of components you calculated above.

# projection to low dimensional space:
# l = ?
proj_v = U[:,:l].T.dot(V.T)
V_tr=proj_v.T

# normalization:
std_V = np.std(V_tr, 0)
std_V[std_V==0] = 1
mean_V = np.mean(V_tr,0)
std_Z = np.std(Z,0)
std_Z[std_Z==0] = 1
mean_Z = np.mean(Z,0)

# setting up the network input and output:
Y_train = (V_tr - mean_V)/std_V
X_train = (Z - mean_Z)/std_Z

# checking the size of input and output:
print(X_train.shape)
print(Y_train.shape)

In [ ]:
def generate_model(input_dim, output_dim, n_layer, n_hidden,
                   optimizer_meth, act_fcn, learning_rate = 0.001, reg=0.01):
    model = Sequential()
    """
    input_dim:           the size of input vector
    output_dim:          the size of output vector
    n_layer:             number of hidden layers in the network
    n_hidden:            a list of number of hidden units in each hidden layer. 
                         for example, for n_hidden=3, this can be [200, 100, 45]
    optimizer_meth:      the optimization method used for training
    act_fcn              the type of activation function in the hidden layers
    learning_rate        Learning rate used in the training phase
    reg                  L2 regularization coefficient
    """
    
    #### ToDo for part 2. Build your model here and compile it. You should copy-paste your code from Problem 1.
    """ hint: Keep the parameters for the hidden layers the same, and use for loop to add the layers to the 
              network. Add the last layer (output layer) outside the for loop since it has a different
              activation function. 
        
    """
    
    return model

In [ ]:
def train(model, x_train, y_train, batch_size, epochs, verbose=1, validation_split=.2):
    """
    model:       the DNN model (the output of generate_model)
    x_train:     input of the network
    y_train:     output of the network
    batch_size:  batch size parameter
    epochs:      number of epochs used to train the network
    validation_split:  validation/train split ration
    """
    
    #### ToDo for part 2. Use the training data and train your model. You should copy-paste your code from Problem 1.

In [ ]:
def hyp_tuning(learning_rate=[1e-4,1e-3,1e-2,1e-1], regularization =[1e-4,1e-3,1e-2,1e-1],n_layer=1,hidden_nodes=50):
    ### ToDo (optional) for part 2. Find the optimal combination of learning rate and regularization coefficient.
    ### You can copy-paste your code from Problem 1.
    
    return opt_lr, opt_reg

In [ ]:
# Don't change this block. It's for post-processing and plotting the results
def post_process_low_rank(data, U):
    nx = [501, 41]
    n_edge = 11
    a = np.zeros((nx[1], nx[0]))
    for i in range(data.shape[0]):
        a[:, i:i+n_edge] += np.dot(U, data[i, :]).reshape(nx[1], -1)
    for i in range(n_edge):
        a[:, i] = a[:, i]/(i+1)
        a[:, nx[0]-i-1] = a[:, nx[0]-i-1]/(i+1)

    a[:, n_edge: nx[0]-n_edge] = a[:, n_edge: nx[0]-n_edge]/n_edge
    return a

def post_process(data):
    nx = [501, 41]
    n_edge = 11
    a = np.zeros((nx[1], nx[0]))
    for i in range(data.shape[0]):
        a[:, i:i+n_edge] += data[i, :].reshape(nx[1], -1)
    for i in range(n_edge):
        a[:, i] = a[:, i]/(i+1)
        a[:, nx[0]-i-1] = a[:, nx[0]-i-1]/(i+1)

    a[:, n_edge: nx[0]-n_edge] = a[:, n_edge: nx[0]-n_edge]/n_edge
    return a

def plt_im_tri(depth, fig_name, show_file=True, vmin_=21, vmax_=29,label_z='Elevation [m]'):
    mesh = loadmat("mesh.mat")
    triangles = mesh['triangles']
    meshnode = mesh['meshnode']
    matplotlib.rcParams.update({'font.size': 16})

    offsetx = 220793.
    offsety = 364110.
    fig_index = 1
    plt.figure(fig_index, figsize=(10., 10.), dpi=100)
    fig_index += 1
    ax = plt.gca()
    im = plt.tripcolor(meshnode[:, 0]*0.3010-offsetx, meshnode[:, 1]*0.3010-offsety,
                       triangles, depth*0.301, cmap=plt.get_cmap('jet'), vmin=vmin_, vmax=vmax_, label='_nolegend_')
    ax.set_xlabel("Easting [m]")
    ax.set_ylabel("Northing [m]")
    plt.gca().set_aspect('equal', adjustable='box')
    plt.axis([0., 1000., 0., 530.])
    plt.xticks(np.arange(0., 1000.+10., 200.0))
    plt.yticks(np.arange(0., 530.+10., 200.0))
    cbar = plt.colorbar(im, fraction=0.025, pad=0.05)
    cbar.set_label(label_z)
    plt.rcParams['axes.axisbelow'] = True
    plt.rc('axes', axisbelow=True)
    plt.grid()
    ax.set_axisbelow(True)
    plt.tight_layout()
    plt.savefig(fig_name)
    if show_file:
        plt.show()

def plot_inverse(model, mean_V=None, std_V=None, mean_Z=None, std_Z=None):
    data_red_river = loadmat('P2_test.mat')
    V = data_red_river['X']
    Z = data_red_river['Y']
    Uz = data_red_river['U']
    V_P = (V-mean_V)/std_V
    pred = model.predict(V_P)
    Z_P = pred*std_Z+mean_Z
    river_prof = post_process_low_rank(Z_P, Uz)
    plt_im_tri(river_prof.ravel(), 'predicted_river.jpg', False,21,29,label_z='Elevation [m]')

def plot_forward_low_rank(model, mean_Z=None, std_Z=None, mean_V=None, std_V=None, U_l=None):
    data_red_river = loadmat('P2_test.mat')
    V = data_red_river['X']
    Z = data_red_river['Y']
    Uz = data_red_river['U']
    Z_P = (Z-mean_Z)/std_Z
    pred = model.predict(Z_P)
    V_P = pred*std_V+mean_V
    velocity_prof = post_process_low_rank(V_P, U_l)
    plt_im_tri(velocity_prof.ravel(), 'predicted_velocity.jpg', False,-0.5,1,label_z='Velocity [m/s]')

def plot_forward(model, mean_Z=None, std_Z=None, mean_V=None, std_V=None):
    data_red_river = loadmat('P2_test.mat')
    V = data_red_river['X']
    Z = data_red_river['Y']
    Uz = data_red_river['U']
    Z_P = (Z-mean_Z)/std_Z
    pred = model.predict(Z_P)
    V_P = pred*std_V+mean_V
    velocity_prof = post_process(V_P)
    plt_im_tri(velocity_prof.ravel(), 'predicted_velocity.jpg', False,-0.5,1,label_z='Velocity [m/s]')

In [ ]:
"""
ToDo:  
Train your network with output dimension=l. You can use your codes from problem 1
"""

In [ ]:
# ToDo: calculate RMSE of validation set of velocity. Note that the network output is 
# low-dimensional velocity after normalization. Make sure you convert these outputs to velocities.


In [ ]:
# Calculation of RMSE of the test data. Here we use post_process function to calculate RMSE on the whole 
# river instead of small segments. You do not need to use post_process for RMSE of validation set asked above 

# load test dataset: 
data_red_river = loadmat('P2_test.mat')
V_test = data_red_river['X']
Z_test = data_red_river['Y']
Uz = data_red_river['U']
Z_P = (Z_test-mean_Z)/std_Z

#predict test set velocity RMSE using your trained network:
pred = model.predict(Z_P)
V_P = pred*std_V+mean_V
velocity_prof = post_process_low_rank(V_P,U[:,:l])
velocity_prof_true = post_process(V_test)
print(np.sqrt(np.mean((velocity_prof-velocity_prof_true)**2))) #RMSE